<a href="https://colab.research.google.com/github/Kyubyong/DeepRL-Agents/blob/master/notebooks/how-to-segment-anything-with-sam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# Segment Anything Model (SAM)

---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/facebookresearch/segment-anything) [![arXiv](https://img.shields.io/badge/arXiv-2304.02643-b31b1b.svg)](https://arxiv.org/abs/2304.02643)

Segment Anything Model (SAM): a new AI model from Meta AI that can "cut out" any object, in any image, with a single click. SAM is a promptable segmentation system with zero-shot generalization to unfamiliar objects and images, without the need for additional training. This notebook is an extension of the [official notebook](https://colab.research.google.com/github/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb) prepared by Meta AI.

![segment anything model](https://media.roboflow.com/notebooks/examples/segment-anything-model-paper.png)

## Complementary Materials

---

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/how-to-segment-anything-with-sam.ipynb) [![YouTube](https://badges.aleen42.com/src/youtube.svg)](https://youtu.be/D-D6ZmadzPE) [![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/how-to-use-segment-anything-model-sam)

We recommend that you follow along in this notebook while reading the blog post on Segment Anything Model.

![segment anything model blogpost](https://media.roboflow.com/notebooks/examples/segment-anything-model-blogpost.png)

## Pro Tip: Use GPU Acceleration

If you are running this notebook in Google Colab, navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`. This will ensure your notebook uses a GPU, which will significantly speed up model training times.

## Steps in this Tutorial

In this tutorial, we are going to cover:

- **Before you start** - Make sure you have access to the GPU
- Install Segment Anything Model (SAM)
- Download Example Data
- Load Model
- Automated Mask Generation
- Generate Segmentation with Bounding Box
- Segment Anything in Roboflow Universe Dataset

## Let's begin!

In [2]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


## Install Segment Anything Model (SAM) and other dependencies

In [3]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

  Preparing metadata (setup.py) ... done


In [4]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision==0.23.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.4 MB/s eta 0:00:00


### Download SAM weights

In [5]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [6]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [7]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [8]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

## Automated Mask Generation

To run automatic mask generation, provide a SAM model to the `SamAutomaticMaskGenerator` class. Set the path below to the SAM checkpoint. Running on CUDA and with the default model is recommended.

In [9]:
mask_generator = SamAutomaticMaskGenerator(sam)

In [11]:
from google.colab import files
files.download('weights/sam_vit_h_4b8939.pth')  # 로컬로 다운로드


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Generate masks with SAM

In [16]:
import zipfile
import os
import glob

# 1. 압축 해제
zip_path = "/content/아카이브.zip"
extract_dir = "/content/images2"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 2. 모든 하위 폴더 포함해서 *.jpg 파일 찾기
png_files = glob.glob(os.path.join(extract_dir, '**', '*.png'), recursive=True)

# 3. 결과 확인
print(f"총 {len(png_files)}개의 JPG 파일을 찾았습니다.")
# for path in jpg_files:
#     print(path)

총 776개의 JPG 파일을 찾았습니다.


In [ ]:

import cv2
import supervision as sv
from tqdm import tqdm
import numpy as np


for png_file in tqdm(sorted(png_files)):
  if "_" in png_file:
    continue
  print(png_file)
  path_name = os.path.dirname(png_file)
  folder_name = png_file.split('images2/', 1)[-1].split('/')[0]

  image_bgr = cv2.imread(png_file, cv2.IMREAD_UNCHANGED)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

  sam_result = mask_generator.generate(image_rgb)
  detections = sv.Detections.from_sam(sam_result=sam_result)

  # 세그먼트된 결과에서 bounding box들과 마스크 가져오기
  boxes = detections.xyxy.astype(int)
  masks = detections.mask  # shape: (N, H, W)

  # 출력 폴더
  # os.makedirs("cropped_segments", exist_ok=True)

  # 이미지 크롭 + 마스킹
  for i, (box, mask) in enumerate(zip(boxes, masks)):
    x1, y1, x2, y2 = box
    cropped_image = image_bgr[y1:y2, x1:x2]
    cropped_mask = mask[y1:y2, x1:x2]

    # 알파 마스크 생성 + 외곽 블러 처리
    alpha = (cropped_mask * 255).astype(np.uint8)
    blurred_alpha = cv2.GaussianBlur(alpha, (9, 9), 0)  # 부드러운 외곽

    # RGB 채널 분리
    if cropped_image.shape[2] == 4:
        b, g, r, _ = cv2.split(cropped_image)
    else:
        b, g, r = cv2.split(cropped_image)

    # RGBA 병합
    rgba = cv2.merge([b, g, r, blurred_alpha])

    # 저장
    save_path = f"{path_name}/{folder_name}_{i+1}.png"
    cv2.imwrite(save_path, rgba, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    import shutil
    import os

    # 대상 폴더 생성
    target_dir = f"/content/drive/MyDrive/images2/{folder_name}"
    os.makedirs(target_dir, exist_ok=True)

    # 파일 복사
    shutil.copy(save_path, target_dir)




  0%|          | 0/776 [00:00<?, ?it/s]

/content/images2/0/1.png


  0%|          | 1/776 [00:07<1:37:42,  7.56s/it]

/content/images2/0/10.png


  0%|          | 2/776 [00:14<1:31:18,  7.08s/it]

...

/content/images2/287/721ba2c3-192e-4300-b9ef-74ec44865062.png


 30%|███       | 233/776 [38:50<1:37:16, 10.75s/it]

/content/images2/288/01439d51-802f-43c7-8344-6c27e0e4f6f9.png


 30%|███       | 234/776 [39:00<1:36:06, 10.64s/it]

/content/images2/289/c8791bac-4fe7-4aea-a875-cb31028d6efd.png


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
